# Create Video Frame Table 2

Create a 3LC Table by extracting all individual frames from video files in the UCF11 action recognition dataset.

![img](../images/create-video-thumbnail-table.jpg)

<!-- Tags: ["video", "frames", "action-recognition", "intermediate"] -->

Video analysis often requires working with individual frames rather than entire video files. This approach allows for frame-level analysis, data augmentation, and easier integration with image-based machine learning pipelines.

This notebook processes video files from the UCF11 dataset, extracting all frames as PIL Images and creating a table where each row represents a single frame. Each frame is linked to its source video and frame number, enabling both individual frame analysis and sequence reconstruction. The dataset follows a structured format with categorized video clips:

```
UCF11/
├─ basketball/
│  ├─ v_shooting_01
|  │  ├─ v_shooting_01_01.mpg
|  │  ├─ v_shooting_01_02.mpg
|  │  ├─ ...
│  ├─ v_shooting_02
|  │  ├─ ...
├─ biking/
│  ├─ ...
├─ ...
```

## Project setup

In [1]:
DATA_PATH = "../../data"
PROJECT_NAME = "3LC Tutorials - Create Tables"
TABLE_NAME = "UCF YouTube Actions - Frames"

## Imports

In [2]:
from pathlib import Path

import cv2
import tlc
from PIL import Image

## Create Table

The class names are read from the directory names.

In [3]:
DATASET_LOCATION = Path(DATA_PATH) / "ucf11"

assert DATASET_LOCATION.exists(), f"Dataset not found at {DATASET_LOCATION}"

In [4]:
class_directories = [path for path in DATASET_LOCATION.glob("*") if path.is_dir()]

In [5]:
classes = [c.name for c in class_directories]
classes

['volleyball_spiking',
 'golf_swing',
 'swing',
 'tennis_swing',
 'soccer_juggling',
 'horse_riding',
 'walking',
 'basketball',
 'trampoline_jumping',
 'biking',
 'diving']

We now define a schema for the `Table`. Each row will contain a sequence ID (video path), frame ID (frame number), the actual frame as a PIL Image, and a categorical label for the video class.

In [7]:
column_schemas = {
    "sequence_id": tlc.StringSchema(),
    "frame_id": tlc.Int32Schema(),
    "frame": tlc.ImageUrlSchema(sample_type="PILImage"),
    "label": tlc.CategoricalLabelSchema(classes),
}

/Users/gudbrand/Projects/tlc-monorepo/src/tlc/core/builtins/schemas/schemas.py:400: UserWarning: class_names is deprecated, use classes instead
  warnings.warn("class_names is deprecated, use classes instead")


We then iterate over the videos, extract all frames as PIL Images, and write the `Table` with a `TableWriter`.

In [ ]:
def extract_frames_from_video(video_path):
    """Extract all frames from a video and return them as PIL Images."""
    cap = cv2.VideoCapture(str(video_path))
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert BGR to RGB (OpenCV uses BGR, PIL expects RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert to PIL Image
        pil_image = Image.fromarray(frame_rgb)
        frames.append(pil_image)
    
    cap.release()
    return frames

table_writer = tlc.TableWriter(
    project_name=PROJECT_NAME,
    dataset_name="UCF YouTube Actions",
    table_name=TABLE_NAME,
    column_schemas=column_schemas,
)

with tlc.bulk_data_url_context(table_writer.url.parent.parent / "bulk_data", table_writer.url):
    for class_idx, class_directory in enumerate(class_directories):
        for video_path in class_directory.rglob("*mpg"):
            video_path = video_path.absolute()
            
            # Extract all frames from the video
            frames = extract_frames_from_video(video_path)
            
            # Get the sequence_id (relative path to video)
            sequence_id = tlc.Url(video_path).to_relative().to_str()
            
            # Write a row for each frame
            for frame_id, frame in enumerate(frames):
                row = {
                    "sequence_id": sequence_id,
                    "frame_id": frame_id,
                    "frame": frame,
                    "label": class_idx,
                }
                table_writer.add_row(row)

table = table_writer.finalize()

In [11]:
table[0]

{'sequence_id': '/Users/gudbrand/Projects/3lc-examples/data/ucf11/volleyball_spiking/v_spiking_01/v_spiking_01_02.mpg',
 'frame_id': 0,
 'frame': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=320x240>,
 'label': 0}

In [12]:
table.table_rows[0]

ImmutableDict({'sequence_id': '/Users/gudbrand/Projects/3lc-examples/data/ucf11/volleyball_spiking/v_spiking_01/v_spiking_01_02.mpg', 'frame_id': 0, 'frame': '/Users/gudbrand/Projects/3lc-examples/tutorials/1-create-tables/value/0000000.png', 'label': 0})

In [13]:
table.url

Url('/Users/gudbrand/Library/Application Support/3LC/projects/3LC Tutorials - Create Tables/datasets/UCF YouTube Actions/tables/UCF YouTube Actions - Frames')